In [44]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import cv2
import os
import tensorflow as tf
import json
import crop


from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler


### _07: 입술건조(lip_dryness)에 대해 bbox_crop

In [5]:
label_key = "_F_07.json"
annotation_key = "lip_dryness"

In [6]:
image_data, target_data = crop.process_files(label_key, annotation_key)

In [7]:
print(f"Number of images processed: {len(image_data)}")
print(f"Number of targets processed: {len(target_data)}")

Number of images processed: 2574
Number of targets processed: 2574


### 데이터 저장 및 불러오기

In [8]:
# 이미지, 타겟 데이터 저장

with open(f'crop_data/{annotation_key}_image_data.pkl', 'wb') as f:
    pickle.dump(image_data, f)

with open(f'crop_data/{annotation_key}_target_data.pkl', 'wb') as f:
    pickle.dump(target_data, f)

In [45]:
# 데이터 로드
with open(f'crop_data/{annotation_key}_image_data.pkl', 'rb') as f:
    image_data = pickle.load(f)

with open(f'crop_data/{annotation_key}_target_data.pkl', 'rb') as f:
    target_data = pickle.load(f)

In [46]:
# 데이터 카운트
from collections import Counter

values_list = list(target_data.values())
value_counts = Counter(values_list)

for value, count in value_counts.items():
    print(f"Value: {value}, Count: {count}")

Value: 2, Count: 1572
Value: 3, Count: 477
Value: 1, Count: 405
Value: 4, Count: 60
Value: 0, Count: 60


### (0,1), (3,4)로 모델링

In [47]:
def merge_classes(target_data):
    binary_target_data = {}
    for key, value in target_data.items():
        if value in [0, 1]:
            binary_target_data[key] = '10'
        elif value in [3, 4]:
            binary_target_data[key] = '20'
        else:
            binary_target_data[key] = value
    
    return binary_target_data

binary_target_data = merge_classes(target_data)

In [48]:
filtered_target_data = {key: value for key, value in binary_target_data.items() if value in ['10', '20']}

values_list = list(filtered_target_data.values())
value_counts = Counter(values_list)

for value, count in value_counts.items():
    print(f"Value: {value}, Count: {count}")

Value: 20, Count: 537
Value: 10, Count: 465


In [49]:
common_keys = set(image_data.keys()) & set(filtered_target_data.keys())

# 데이터 정렬
filtered_image_data = {key: image_data[key] for key in common_keys}
filtered_target_data = {key: filtered_target_data[key] for key in common_keys}

# X와 y 생성
X = np.array(list(filtered_image_data.values()))
y = np.array(list(filtered_target_data.values()))

print(f"Length of X: {len(X)}, Length of y: {len(y)}")

Length of X: 1002, Length of y: 1002


In [50]:
class_mapping = {'10': 0, '20': 1}
y = np.array([class_mapping[label] for label in y])
X = X / 255.0

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_train shape: (801, 128, 128, 3), y_train shape: (801,)
X_test shape: (201, 128, 128, 3), y_test shape: (201,)


In [51]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X.shape[1], X.shape[2], X.shape[3])),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # 이진 분류 출력층
])

# 모델 컴파일
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(learning_rate=0.0001), 
              metrics=['accuracy'])

# 모델 요약
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 63, 63, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 30, 30, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_11 (Conv2D)          (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 14, 14, 128)      

In [52]:
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)


In [53]:
# 모델 학습
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    class_weight = class_weights,
    callbacks=[early_stopping]
)

Epoch 1/100
20/20 [==============================] - 6s 221ms/step - loss: 0.7033 - accuracy: 0.4781 - val_loss: 0.6926 - val_accuracy: 0.5280
Epoch 2/100
20/20 [==============================] - 4s 212ms/step - loss: 0.6930 - accuracy: 0.5078 - val_loss: 0.6922 - val_accuracy: 0.5280
Epoch 3/100
20/20 [==============================] - 4s 208ms/step - loss: 0.6954 - accuracy: 0.4578 - val_loss: 0.6928 - val_accuracy: 0.5280
Epoch 4/100
20/20 [==============================] - 4s 206ms/step - loss: 0.6931 - accuracy: 0.5156 - val_loss: 0.6929 - val_accuracy: 0.5217
Epoch 5/100
20/20 [==============================] - 4s 204ms/step - loss: 0.6929 - accuracy: 0.5328 - val_loss: 0.6929 - val_accuracy: 0.4907
Epoch 6/100
20/20 [==============================] - 4s 205ms/step - loss: 0.6926 - accuracy: 0.5188 - val_loss: 0.6929 - val_accuracy: 0.4720
Epoch 7/100
20/20 [==============================] - 4s 202ms/step - loss: 0.6939 - accuracy: 0.4953 - val_loss: 0.6928 - val_accuracy: 0.4720

In [54]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

7/7 - 0s - loss: 0.5816 - accuracy: 0.7164 - 310ms/epoch - 44ms/step
Test Loss: 0.5816039443016052
Test Accuracy: 0.7164179086685181


In [55]:
# 예측 수행 (확률 값 반환)
predictions = model.predict(X_test)  # 이진 분류에서는 sigmoid 출력

# 0.5를 기준으로 클래스 결정
predicted_classes = (predictions > 0.5).astype(int).flatten()

print("Predicted Classes: ", predicted_classes)


7/7 [==============================] - 0s 42ms/step
Predicted Classes:  [1 1 1 1 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0 0 0 1 1 1 1 1 1 0
 0 0 1 1 0 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0
 1 1 0 0 1 1 1 1 0 1 1 1 0 1 1 0 1 1 1 1 0 0 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1
 1 0 0 0 1 1 1 1 1 1 0 1 1 0 0 1 1 0 1 1 1 1 0 1 1 1 0 0 1 0 0 0 1 0 0 1 1
 0 0 1 1 1 0 1 0 0 1 1 1 1 1 1 1 0 1 1 1 0 0 0 1 1 0 1 1 1 0 1 1 0 1 1 1 1
 0 1 0 1 0 0 0 0 1 1 1 1 1 1 0 1]
